In [35]:
import pandas as pd
import numpy as np 
import networkx as nx
import pickle as pkl

dt = 'FinCausal'

df = pd.read_csv(dt + '.csv')

df

,Text,Cause,Effect,Label
0,The increase in net interest income was due pr...,a $152.9 billion increase in average outstandi...,The increase in net interest income,causal
1,"NPAs increased $703 million year over year, pr...",PCI loans that would have been classified as n...,NPAs increased $703 million year over year,causal
2,Additional increases in noninterest income wer...,higher insurance income driven by improved pro...,Additional increases in noninterest income,causal
3,"During 2020, the Company also issued $6.5 bill...",The increase is due to net income in excess of...,"During 2020, the Company also issued $6.5 bill...",causal
4,The significant increases in earnings assets a...,"the Merger, as well as impacts from the COVID-...",The significant increases in earnings assets a...,causal
...,...,...,...,...
1073,-,Interest expense was $15.8 million and $20.7 m...,the $159 million environmental remediation cha...,non_causal
1074,-,The decrease in net income (loss) attributable...,lost distribution in certain product categorie...,non_causal
1075,-,"In constant currency, SG&A costs as a percenta...",Project Renewal and the European Transformatio...,non_causal
1076,-,The yield on the total loan portfolio for the ...,the Merger and growth in commercial and indust...,non_causal


In [41]:
graph = nx.Graph()

cause = ' Cause'
effect = ' Effect'
# cause = 'left_title'
# effect = 'right_title'

for _, row in df.iterrows(): 
    #if row.simple_result != 'not_cause':
    graph.add_edge(row[cause], row[effect])

In [37]:
print(len(graph.nodes))
print(len(graph.edges))

860
991


In [38]:
from plotly import graph_objs as go

def show_graph(G):
  ### ARESTAS
  edge_x = []
  edge_y = []

  # adicionando as coordenadas
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  # definindo cor e estilo das arestas
  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=2, color='#888'),
      hoverinfo='none',
      mode='lines')

  ### VÉRTICES
  node_x = []
  node_y = []

  # adicionando as coordenadas
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)

  # definindo cor e estilo dos vértices
  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          size=10,
          line_width=2))

  # visualizando!
  fig = go.Figure(data=[edge_trace, node_trace],
              layout=go.Layout(
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

In [43]:
pos = nx.spring_layout(graph,seed=81) # obtém coordenadas dos vértices para visualização
for node in graph.nodes():
  graph.nodes[node]['pos'] = pos[node]

In [44]:
show_graph(graph)

In [42]:
nx.number_connected_components(graph)

51